TRABALHO BD

A partir de webscrapping, ir buscar o número de ids escolhido (máx 20) de termos à escolha do utilizador e importar as informações para a base de dados

WEBSCRAPPING

In [730]:
import re

def validateInsertedText(item):
    return re.match(r'[\d|a-z|A-Z]+(\_?[\d|a-z|A-Z]+)*', item)

def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(item)
    url_list.append(url)
    return url_list

query = input("Defina o termo que pretende pesquisar [bacteria/yeast/human...] \n Se o termo tiver mais do que uma palavra, separe por _: ")
try:
    
    if not validateInsertedText(query):
        raise Exception
    else:   
        print(url_get(query))

except Exception:
    print(f'Failed to obtain erro .l. next time be a goob boy :). This is what you inserted:"{query}"')


['https://www.ncbi.nlm.nih.gov/gene/?term=a ']


In [713]:
import requests, sys, json
content = []
for url in url_get(query):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [714]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [715]:
import re
existe = re.findall(r'\s*ID:\s*\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = re.sub(r'\s*ID:\n*\s+', '', id_genes)
id_gene = id_g.split(',')
id_gene


['9615',
 '249359',
 '403879',
 '442858',
 '403811',
 '403869',
 '403802',
 '475738',
 '442942',
 '403885',
 '403424',
 '474860',
 '489652',
 '403418',
 '403980',
 '804479',
 '610255',
 '403803',
 '477032',
 '403955',
 '404306']

In [719]:
try:
    number_of_genes = int(input("Número de genes a obter (máx: 20): "))
    print(number_of_genes)
    genes = id_gene[0:number_of_genes]
    print(genes)
except Exception:
    print("INVALID")

INVALID


In [7]:
#BUSCAR LINKS DOS GENES

In [661]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/56784',
 'https://www.ncbi.nlm.nih.gov/nuccore/66380993',
 'https://www.ncbi.nlm.nih.gov/nuccore/66380996',
 'https://www.ncbi.nlm.nih.gov/nuccore/8912104',
 'https://www.ncbi.nlm.nih.gov/nuccore/8912103',
 'https://www.ncbi.nlm.nih.gov/nuccore/69752209',
 'https://www.ncbi.nlm.nih.gov/nuccore/66380995',
 'https://www.ncbi.nlm.nih.gov/nuccore/60851505',
 'https://www.ncbi.nlm.nih.gov/nuccore/64220641']

In [662]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [663]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
#r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
locus_list = []
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')

	if "Error" not in r2.content.decode('utf-8'):
		locus_list.append(r2.content.decode('utf-8'))
print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

LOCUS       X67302                   369 bp    mRNA    linear   ROD 26-JUL-2016
DEFINITION  R.norvegicus mRNA for Oct 2 Pou-homeodomain protein.
ACCESSION   X67302
VERSION     X67302.1
KEYWORDS    homeodomain protein; Oct-2 gene; POU domain protein.
SOURCE      Rattus norvegicus (Norway rat)
  ORGANISM  Rattus norvegicus
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Glires; Rodentia; Myomorpha;
            Muroidea; Muridae; Murinae; Rattus.
REFERENCE   1  (bases 1 to 369)
  AUTHORS   Briton-Jones,C.M. and Rodgers,R.J.
  TITLE     Cloning and sequencing of POU-Homeo domain gene expressed in rat
            ovaries
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 369)
  AUTHORS   Rodgers,R.J.
  TITLE     Direct Submission
  JOURNAL   Submitted (07-JUL-1992) R.J. Rodgers, Flinders Univ. of South
            Australia, Dept. of Medicine, Bedford Park, SA 5042, AUSTRALIA
FEATURES             Location/Qualif

LINK NCBI

In [664]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista


['https://www.ncbi.nlm.nih.gov/nuccore/X67302',
 'https://www.ncbi.nlm.nih.gov/nuccore/BW778499',
 'https://www.ncbi.nlm.nih.gov/nuccore/BW778502',
 'https://www.ncbi.nlm.nih.gov/nuccore/BB232249',
 'https://www.ncbi.nlm.nih.gov/nuccore/BB232248',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAHY01223610',
 'https://www.ncbi.nlm.nih.gov/nuccore/BW778501',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAGJ01210774',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAGV01843290']

ACCESSION

In [665]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['X67302 ',
 'BW778499 ',
 'BW778502 ',
 'BB232249 ',
 'BB232248 ',
 'AAHY01223610 AAHY01000000 ',
 'BW778501 ',
 'AAGJ01210774 AAGJ01000000 ',
 'AAGV01843290 AAGV01000000 ']

VERSION

In [666]:
version_list = []

existe = re.findall(r'VERSION.*?(?=KEYWORDS|DBLINK)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for vers in existe:
        m = re.match( r'VERSION\s+(.+)', vers, re.DOTALL )
        version_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
version_list

['X67302.1 ',
 'BW778499.1 ',
 'BW778502.1 ',
 'BB232249.1 ',
 'BB232248.1 ',
 'AAHY01223610.1 ',
 'BW778501.1 ',
 'AAGJ01210774.1 ',
 'AAGV01843290.1 ']

DEFINITION

In [667]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['R.norvegicus mRNA for Oct 2 Pou-homeodomain protein. ',
 "BW778499 Amphioxus Branchiostoma floridae unpublished cDNA library, gastrula whole animal Branchiostoma floridae cDNA clone bfga051e13 5', mRNA sequence. ",
 "BW778502 Amphioxus Branchiostoma floridae unpublished cDNA library, gastrula whole animal Branchiostoma floridae cDNA clone bfga051e16 5', mRNA sequence. ",
 "BB232249 RIKEN full-length enriched, 3 days neonate thymus Mus musculus cDNA clone A630036M17 3' similar to AF236887 Mus musculus protein kinase ATR (Atr) mRNA, mRNA sequence. ",
 "BB232248 RIKEN full-length enriched, 3 days neonate thymus Mus musculus cDNA clone A630036M16 3', mRNA sequence. ",
 'Mus musculus strain 129X1/SvJ chromosome Un CRA_211000020833787, whole genome shotgun sequence. ',
 "BW778501 Amphioxus Branchiostoma floridae unpublished cDNA library, gastrula whole animal Branchiostoma floridae cDNA clone bfga051e15 5', mRNA sequence. ",
 'Strongylocentrotus purpuratus isolate A3 Spur_1.1_Reptig6721, w

LOCUS

In [668]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['X67302 369 bp mRNA linear ROD 26-JUL-2016 ',
 'BW778499 339 bp mRNA linear EST 04-JAN-2011 ',
 'BW778502 451 bp mRNA linear EST 04-JAN-2011 ',
 'BB232249 291 bp mRNA linear EST 04-JUL-2000 ',
 'BB232248 297 bp mRNA linear EST 04-JUL-2000 ',
 'AAHY01223610 742 bp DNA linear ROD 07-JUL-2005 ',
 'BW778501 336 bp mRNA linear EST 04-JAN-2011 ',
 'AAGJ01210774 910 bp DNA linear INV 11-MAR-2005 ',
 'AAGV01843290 1143 bp DNA linear MAM 13-MAY-2005 ']

LOCUS NAME

In [669]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list


['X67302',
 'BW778499',
 'BW778502',
 'BB232249',
 'BB232248',
 'AAHY01223610',
 'BW778501',
 'AAGJ01210774',
 'AAGV01843290']

NUMBER OF BP

In [670]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 

#cria lista com os valores do locus, e escolhe os 3º e 4º caracteres, que correspondem ao nr de bp
loc_bp_list

['369 bp',
 '339 bp',
 '451 bp',
 '291 bp',
 '297 bp',
 '742 bp',
 '336 bp',
 '910 bp',
 '1143 bp']

MOLECULE TYPE

In [671]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))
#temos que corrigir esta porque as mol types podem ser de uma, duas ou trÊs palavras..
mol_type

['mRNA linear',
 'mRNA linear',
 'mRNA linear',
 'mRNA linear',
 'mRNA linear',
 'DNA linear',
 'mRNA linear',
 'DNA linear',
 'DNA linear']

GENBANK DIVISION

In [672]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['ROD', 'EST', 'EST', 'EST', 'EST', 'ROD', 'EST', 'INV', 'MAM']

MODIFICATION DATE

In [673]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['26-JUL-2016',
 '04-JAN-2011',
 '04-JAN-2011',
 '04-JUL-2000',
 '04-JUL-2000',
 '07-JUL-2005',
 '04-JAN-2011',
 '11-MAR-2005',
 '13-MAY-2005']

SOURCE

In [674]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['Rattus norvegicus (Norway rat) ',
 'Branchiostoma floridae (Florida lancelet) ',
 'Branchiostoma floridae (Florida lancelet) ',
 'Mus musculus (house mouse) ',
 'Mus musculus (house mouse) ',
 'Mus musculus (house mouse) ',
 'Branchiostoma floridae (Florida lancelet) ',
 'Strongylocentrotus purpuratus ',
 'Dasypus novemcinctus (nine-banded armadillo) ']

ORGANISM

In [675]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Rattus norvegicus',
 'Branchiostoma floridae',
 'Branchiostoma floridae',
 'Mus musculus',
 'Mus musculus',
 'Mus musculus',
 'Branchiostoma floridae',
 'Strongylocentrotus purpuratus',
 'Dasypus novemcinctus']

FEATURES - CDS

In [676]:
from Bio import SeqIO

In [677]:
from Bio import Entrez
from Bio import SeqIO
from Bio import SeqFeature
from Bio.SeqFeature import SeqFeature, FeatureLocation

Entrez.email = "pg49849@alunos.uminho.pt"

features=[]
CDS=[] 
source = []
gene = []


for acc in acc_list:
    handle = Entrez.efetch(db="nucleotide", rettype = "gb", retmode= "text", id=acc)
    
    for record in SeqIO.parse(handle,"gb"):
        
        for feature in record.features:
            features.append(feature.type)

        aux_source = []
        aux_gene = []


        for feature in record.features:
            if feature.type == "source":
                aux_source.append(str(feature.location))

            if feature.type == 'gene':
                aux_gene.append(feature.qualifiers["gene"])
 
        gene.append(aux_gene)
        source.append(aux_source)

        record_CDS = []
        for feature in record.features:
            CDS_aux=[]
            if feature.type == "CDS":
                CDS_aux.append(str(feature.location))
                CDS_aux.append(feature.qualifiers["product"])
                CDS_aux.append(feature.qualifiers["protein_id"])
                CDS_aux.append(feature.qualifiers["translation"])
                # print('->' , CDS_aux)
            record_CDS.append(CDS_aux)
            
        CDS.append(record_CDS)

# print(source)
# print(gene)
# print(exon)
# print(intron)
print(features)
print('*',source)
print('->',gene)
print(len(CDS))
print('p', CDS)  
print('->', CDS) 

['source', 'gene', 'CDS', 'misc_feature', 'misc_feature', 'misc_feature', 'source', 'source', 'source', 'source', 'source', 'source', 'source', 'source']
* [['[0:369](+)'], ['[0:339](+)'], ['[0:451](+)'], ['[0:291](+)'], ['[0:297](+)'], ['[0:742](+)'], ['[0:336](+)'], ['[0:910](+)'], ['[0:1143](+)']]
-> [[['Oct 2']], [], [], [], [], [], [], [], []]
9
p [[[], [], ['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'], ['FTQGDVGLAMGKLYGNDFSQTTISRFEALNLSFKNMCKLKPLLEKWLNDAETMSVDSSLPSPNQLSSPSLGFDGLPGRRRKKRTSIETNVRFALEKSFLANQKPTSEEILLIAEQLHMEKEVI']], [], [], []], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]]]
-> [[[], [], ['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'], ['FTQGDVGLAMGKLYGNDFSQTTISRFEALNLSFKNMCKLKPLLEKWLNDAETMSVDSSLPSPNQLSSPSLGFDGLPGRRRKKRTSIETNVRFALEKSFLANQKPTSEEILLIAEQLHMEKEVI']], [], [], []], [[]], [[]], [[]], [[]], [[]], [[]], [[]], [[]]]


SEQUENCE/ORIGIN

In [678]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['ttcacacagggtgatgtgggcctggccatgggcaagctctatggcaacgacttcagccaaacgaccatttcccgcttcgaggcccttaacctgagcttcaagaacatgtgtaaactcaagcccctcctggagaagtggctcaacgacgcagagactatgtctgtggattcaagcctacccagcccaaaccagctgagcagccccagcctgggtttcgacgggctgccggggcggagacgcaagaagaggaccagcatcgagacgaatgtccgcttcgccttagagaagagtttcctagcgaaccagaagcctacctcagaggagatcctgctgatcgcagagcagctgcacatggaaaaggaagtgatc',
 'atctataacacctaccaatggcggacgcacctccagccggtaggggaggttttcgcggcgggttcggggatcgcggcaggggccgtggacggggccgaggtcgcggcaggggccgtggacggggccgcggaggcaaggaggaaaaggagtggatgccaatgaccaagcttggtcgtcttgtgaaagacctgaagatcaagtccctggaggatatctacctgttctcacttcccatcaaggagtttgagatcattgactttttcctgggttcggcactgaaggatgaggtgctgaagatcatgcctgttcagaagcagacccgtgccggtcagcgcacca',
 'tctaagaacattgtaaggaatcacagcttgtatctccgaatctaagacgtcttcgttatacagagtctcgggaccacaactggtaaacaaagatgtcatcagaagaagaagactacctgtcccccaggtccacaagtatcagtaacggggagagcagtttctacaccagcgctcaaacgcacctgtctgactcacaggacgacgcagacctgaagttctcaggccggtccaagctgagtgaagacagtattgatgagtttatgagcagctgtggaaccta

In [679]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[97, 105, 65, 102],
 [73, 83, 66, 117],
 [131, 116, 98, 106],
 [85, 67, 90, 49],
 [83, 62, 100, 52],
 [336, 102, 136, 168],
 [121, 72, 66, 77],
 [275, 234, 230, 171],
 [312, 272, 321, 238]]

In [680]:
length = []
for seq in seq_origin:
    length.append(len(seq))
length

[369, 339, 451, 291, 297, 742, 336, 910, 1143]

JOURNAL

In [681]:
journ_list = []
existe = re.findall(r'JOURNAL\s+.*?(?=PUBMED|REFERENCE|FEATURES|COMMENT)', locus, re.DOTALL)
if existe:
    for journal in existe:
        m = re.match( r'JOURNAL\s+(.+)', journal, re.DOTALL )
        journ_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
journ_list
#| dá-nos operador ou


['Unpublished ',
 'Submitted (07-JUL-1992) R.J. Rodgers, Flinders Univ. of South Australia, Dept. of Medicine, Bedford Park, SA 5042, AUSTRALIA ',
 'Nature 445 (7128), 613-617 (2007) ',
 'Nature 445 (7128), 613-617 (2007) ',
 'Unpublished (2000) ',
 'Unpublished (2000) ',
 'Science 296 (5573), 1661-1671 (2002) ',
 'Submitted (02-JUN-2005) Celera Genomics, 45 W. Gude Dr., Rockville, MD 20850, USA ',
 'Nature 445 (7128), 613-617 (2007) ',
 'Submitted (14-FEB-2005) Human Genome Sequencing Center, Baylor College of Medicine, One Baylor Plaza, Houston, TX 77030, USA ',
 'Unpublished ',
 'Submitted (05-MAY-2005) Broad Institute of MIT and Harvard, 320 Charles Street, Cambridge, MA 02141, USA ']

AUTHORS

In [475]:
authors_list = []
existe = re.findall(r'AUTHORS\s+.*?(?=TITLE)', locus, re.DOTALL)
if existe:
    for auth in existe:
        m = re.match( r'AUTHORS\s+(.+)', auth, re.DOTALL )
        authors_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
authors_list

['McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and Maier,U.G. ',
 'McFadden,G.I. ',
 'Lang,G., Wotton,D., Owen,M.J., Sewell,W.A., Brown,M.H., Mason,D.Y., Crumpton,M.J. and Kioussis,D. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Satou,Y., Shin-i,T., Kohara,Y. and Satoh,N. ',
 'Buell,C.R., Hart,A., Zismann,V., Karamycheva,S.A., Day,B., Staskawicz,B., Jin,H. and Baker,B. ',
 'Zhao,S., Adams,M.D., Nierman,W., Malek,J., de Jong,P. and Venter,J.C. ',
 'Venter,J.C., Adams,M.D., Myers,E.W., Li,P.W., Mural,R.J., Sutton,G.G., Smith,H.O., Yandell,M., Evans,C.A., Holt,R.A., Gocayne,J.D., Amanatides,P., Ballew,R.M., Huson,D.H., Wortman,J.R., Zhang,Q., Kodira,C.D., Zheng,X.H., Chen,L., Skupski,M., Subramanian,G., Thomas,P.D., Zhang,J., Gabor Miklos,G.L., Nelson,C., Broder,S., Clark,A.G., Nadeau,J., McKusick,V.A., Zinder,N., Levine,A.J., Roberts,R.J., Simon,M., Slayman,C., Hunkapiller,M., Bolanos,R., Delcher,A., Dew

TITLE

In [476]:
titles = []
existe = re.findall(r'TITLE\s+.*?(?=JOURNAL)', locus, re.DOTALL)
if existe:
    for title in existe:
        m = re.match( r'TITLE\s+(.+)', title, re.DOTALL )
        titles.append(re.sub(r'\s+', ' ', m.group(1) ) )
titles

['Evidence that an amoeba acquired a chloroplast by retaining part of an engulfed eukaryotic alga ',
 'Direct Submission ',
 'The structure of the human CD2 gene and its expression in transgenic mice ',
 'Microbial populations from the subantarctic marine ecosystem ',
 'Direct Submission ',
 'Expressed genes in Ciona intestinalis (2002c) ',
 'Generation of EST sequences from Nicotiana benthamiana ',
 'Use of BAC End Sequences from Library RPCI-11 for Sequence-Ready Map Building ',
 'The sequence of the human genome ',
 'Direct Submission ',
 'The sequence of the human genome ',
 'Direct Submission ',
 'The dog genome: survey sequencing and comparative analysis ',
 'Direct Submission ']

PUBMED

In [682]:
pub_list = []
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE|COMMENT)', locus, re.DOTALL)
# print(existe)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)?(?=\w)', pubm, re.DOTALL )
        pub_list.append(re.sub(r'\s+\w+.*', ' ', m.group(1) ) )
print(pub_list)

#procurar os ficheiros que tem o campo PUBMED, se não tiver vai valor vazio na lista (para fazer a relação genbank-PM)
file_locus_pub_list = []
for x in locus_list:
    aux_pub_list = []
    existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE|COMMENT)', x, re.DOTALL)
    # print(existe)
    if existe:
        for pubm in existe:
            m = re.match( r'PUBMED\s+(.+)?(?=\w)', pubm, re.DOTALL )

            aux_pub_list.append(re.sub(r'\s+\w+.*', ' ', m.group(1) ) )
    else:
        aux_pub_list.append('')
    file_locus_pub_list.append(aux_pub_list)

print(file_locus_pub_list)

['1723776', '1723776', '12040188 ', '1723776']
[[''], ['1723776'], ['1723776'], [''], [''], ['12040188 '], ['1723776'], [''], ['']]


In [683]:
import requests
from bs4 import BeautifulSoup

def url_get_id_pm(item):
    url_id_pm = []
    for id_pm in item:
        url = f"https://pubmed.ncbi.nlm.nih.gov/{id_pm}/"
        url_id_pm.append(url)
    return url_id_pm

pm = ""
soup = []
content_pm = []
urls = url_get_id_pm(pub_list)
print(urls)

for url in urls:
	r2 = requests.get(url)
	pm += r2.content.decode('utf-8')
	soup.append(BeautifulSoup(r2.content, 'html.parser'))
	content_pm.append(r2.content.decode('utf-8'))

#print(soup)
#print(content_pm)




['https://pubmed.ncbi.nlm.nih.gov/1723776/', 'https://pubmed.ncbi.nlm.nih.gov/1723776/', 'https://pubmed.ncbi.nlm.nih.gov/12040188 /', 'https://pubmed.ncbi.nlm.nih.gov/1723776/']


AUTHOR, JOURNAL, TITLE, AFFILIATION

In [684]:
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np

In [685]:
Ids=[]
title_list = []
abstract = []
author = []
affiliation = []
journal = []
database = 'PubMed'
Entrez.email= "pg49836@alunos.uminho.pt"
# idlist= pub_list
idlist = []
handle = Entrez.efetch(db=database, id=pub_list, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    title_list.append(info.get("TI", "-"))
    author.append(info.get("AU", "-"))
    journal.append(info.get("SO", "-"))
    affiliation.append(info.get("AD", "-") )
    abstract.append(info.get("AB", "-"))
    print()
print(title_list)
print(abstract)
print(author)
print(affiliation)
print(journal)




['A comparison of whole-genome shotgun-derived mouse chromosome 16 and the human genome.', 'Comparison of three intracellular markers for combined electrophysiological, morphological and immunohistochemical analyses.']
['The high degree of similarity between the mouse and human genomes is demonstrated through analysis of the sequence of mouse chromosome 16 (Mmu 16), which was obtained as part of a whole-genome shotgun assembly of the mouse genome. The mouse genome is about 10% smaller than the human genome, owing to a lower repetitive DNA content. Comparison of the structure and protein-coding potential of Mmu 16 with that of the homologous segments of the human genome identifies regions of conserved synteny with human chromosomes (Hsa) 3, 8, 12, 16, 21, and 22. Gene content and order are highly conserved between Mmu 16 and the syntenic blocks of the human genome. Of the 731 predicted genes on Mmu 16, 509 align with orthologs on the corresponding portions of the human genome, 44 are 

CONNECTION TO DATABASE

In [686]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `LOCUS` (
        `Id_locus` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Accession` VARCHAR(100) NOT NULL,
        `Sequence_length` INT NULL,
        `Molecule_type` VARCHAR(20) NULL,
        `GB_division` VARCHAR(10) NULL,
        `Source` VARCHAR(200) NULL,
        `Organism` VARCHAR(200) NULL,
        `Locus_name` VARCHAR(45) NULL
        );
    """
    Cursor.execute(TableName)
    sql_loc = "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES"
    loc_name_list = list(dict.fromkeys(loc_name_list))  #retirar duplicados 
    for index in range(len(loc_name_list)):
        #fazer a verificação se existe locus name repetido (evitar duplicados)
        select_ids_loc = f"SELECT Id_Locus FROM LOCUS WHERE Locus_name = \"{loc_name_list[index]}\""
        Cursor.execute(select_ids_loc)
        myresult_locus = Cursor.fetchall()
        id_loc = None
        for res_loc, in myresult_locus:
            id_loc = res_loc
        if id_loc is None: 
            if index == len(loc_name_list)-1:
                sql_loc += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\")"
            else:
                sql_loc += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\"),"
    print(sql_loc)
    if sql_loc != "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES":  #não executa nada se não forem dados valores diferentes dos já inseridos
        if sql_loc.endswith(","):  #remove ultimo caracter se este for uma ","
            sql_loc = sql_loc[:-1] #remove utimo caracter da string
        Cursor.execute(sql_loc)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES ("X67302", "369", "mRNA linear", "ROD", "Rattus norvegicus (Norway rat) ", "Rattus norvegicus", "X67302 "), ("BW778499", "339", "mRNA linear", "EST", "Branchiostoma floridae (Florida lancelet) ", "Branchiostoma floridae", "BW778499 "), ("BW778502", "451", "mRNA linear", "EST", "Branchiostoma floridae (Florida lancelet) ", "Branchiostoma floridae", "BW778502 "), ("BB232249", "291", "mRNA linear", "EST", "Mus musculus (house mouse) ", "Mus musculus", "BB232249 "), ("BB232248", "297", "mRNA linear", "EST", "Mus musculus (house mouse) ", "Mus musculus", "BB232248 "), ("AAHY01223610", "742", "DNA linear", "ROD", "Mus musculus (house mouse) ", "Mus musculus", "AAHY01223610 AAHY01000000 "), ("BW778501", "336", "mRNA linear", "EST", "Branchiostoma floridae (Florida lancelet) ", "Branchiostoma floridae", "BW778501 "), ("AAGJ01210774", "910", "DNA linear", "INV", "Strongylocentrotus pu

In [687]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `GENBANK` (
        `Id_GB` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `GB_id` VARCHAR(45) NOT NULL,
        `Definition` VARCHAR(400) NULL,
        `Modification_date` VARCHAR(11) NULL,
        `Version` VARCHAR(20) NOT NULL UNIQUE,
        `LOCUS_Id_locus` INT NOT NULL,
        `ORIGIN_Id_origin` INT NULL
    )
    """
    Cursor.execute(TableName)
    sql_gb = "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES"
    

    for index in range(len(version_list)):
        sql_insert_origin = "INSERT INTO ORIGIN (Sequence, Length, Adenine, Citosine, Timine , Guanine) VALUES"
        select_ids_origin = f"SELECT Id_Origin FROM ORIGIN WHERE Sequence = \"{seq_origin[index]}\""
        Cursor.execute(select_ids_origin)
        myresult_origin = Cursor.fetchall()
        id_ori = None
        for res_ori, in myresult_origin:
            id_ori = res_ori

        if id_ori is None: 
            if index == len(loc_name_list)-1:
                sql_insert_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
            else:
                sql_insert_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\"),"

        if sql_insert_origin != "INSERT INTO ORIGIN (Sequence, Length, Adenine, Citosine, Timine , Guanine) VALUES":
            # select_ids_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
            if sql_insert_origin.endswith(","):  #remove ultimo caracter se este for uma ","
                sql_insert_origin = sql_insert_origin[:-1] #remove utimo caracter da string
            Cursor.execute(sql_insert_origin)

            # print(Cursor.lastrowid)  #buscar o id do origin inserido
            id_inserted_origin = Cursor.lastrowid
        else:
            id_inserted_origin = id_ori


        #fazer a verificação se existe locus repetido (evitar duplicados)
        select_locus = f"SELECT Id_locus from LOCUS where Locus_name = '{loc_name_list[index]}'"
        Cursor.execute(select_locus)
        myresult = Cursor.fetchall()
        id_locus = None
        for res, in myresult:
            id_locus = res
        
        if id_locus is not None:
            select_gb = f"SELECT Id_GB from GENBANK where GB_Id = '{loc_name_list[index]}'"
            Cursor.execute(select_gb)
            myresult_gb = Cursor.fetchall()
            id_genbank = None
            for res_gb, in myresult_gb:
                id_genbank = res_gb
            
            if id_genbank is None:
                if index == len(version_list)-1:
                    sql_gb += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\")"
                else:
                    sql_gb += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\"),"
                
    if sql_gb != "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES":

        if sql_gb.endswith(","):  #remove ultimo caracter se este for uma ","
            sql_gb = sql_gb[:-1] #remove utimo caracter da string
            
        Cursor.execute(sql_gb)

        DataBase.commit()

        print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

9 record inserted.


In [688]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `PUBMED` (
        `Id_PM` INT NOT NULL AUTO_INCREMENT,
        `PM_ID` INT NOT NULL,
        `Journal` VARCHAR(100) NULL,
        `Title` VARCHAR(500) NULL,
        `Abstract` LONGTEXT NULL,
        PRIMARY KEY (`Id_PM`)
    )
    """
    Cursor.execute(TableName)
    
    #print(journal)
    #print(len(title_list))
    #print(len(abstract))
    #print(len(pub_list))
    sql_pm = "INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES"
    pub_list = list(dict.fromkeys(pub_list))  #tirar os duplicados (também n apanha o ultimo)
    #print(pub_list)
    for index in range(len(pub_list)):
        select_pm = f"SELECT Id_PM from PUBMED where PM_Id = '{pub_list[index]}'"
        Cursor.execute(select_pm)
        myresult_pubmed = Cursor.fetchall()
        id_pubmed = None
        for res_pm, in myresult_pubmed:
            id_pubmed = res_pm
        
        if id_pubmed is None:
            if index == len(pub_list)-1:
                sql_pm += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\")"
            else:
                sql_pm += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\"),"
        
    if sql_pm != "INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES":
        # select_ids_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
        if sql_pm.endswith(","):  #remove ultimo caracter se este for uma ","
            sql_pm = sql_pm[:-1] #remove utimo caracter da string
        print(sql_pm)
        Cursor.execute(sql_pm)

        DataBase.commit()

        print(Cursor.rowcount, "record inserted.")

    Cursor.close()
    print(pub_list)

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES ("1723776", "Science. 2002 May 31;296(5573):1661-71. doi: 10.1126/science.1069193.", "A comparison of whole-genome shotgun-derived mouse chromosome 16 and the human genome.", "The high degree of similarity between the mouse and human genomes is demonstrated through analysis of the sequence of mouse chromosome 16 (Mmu 16), which was obtained as part of a whole-genome shotgun assembly of the mouse genome. The mouse genome is about 10% smaller than the human genome, owing to a lower repetitive DNA content. Comparison of the structure and protein-coding potential of Mmu 16 with that of the homologous segments of the human genome identifies regions of conserved synteny with human chromosomes (Hsa) 3, 8, 12, 16, 21, and 22. Gene content and order are highly conserved between Mmu 16 and the syntenic blocks of the human genome. Of the 731 predicted genes on Mmu 16, 509 align with orthologs on the corresponding portions of the human ge

In [689]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)
    print(len(file_locus_pub_list))
    print(len(loc_name_list))

    for index_ids in range(len(file_locus_pub_list)):
        '''for index in range(len(author[index_ids])):
            #verificar se existe autor (evitar duplicados)
            select_ids_auth = f"SELECT Id_Author FROM AUTHORS WHERE Name = \"{author[index_ids]}\""
            Cursor.execute(select_ids_auth)
            myresult_auth = Cursor.fetchall()
            id_auth = None
            for res, in myresult_auth:
                id_affiliation = res'''

        if file_locus_pub_list[index_ids][0] != '':
            db_id_pm = []
            for pm_ids in file_locus_pub_list[index_ids]:
                select_pm_gb = f"SELECT Id_PM FROM PUBMED WHERE PM_ID = \"{pm_ids}\""
                
                #print(select_pm_gb)
                Cursor.execute(select_pm_gb)
                myresult_pm = Cursor.fetchall()
                id_pm = None
                for res, in myresult_pm:
                    id_pm = res
                db_id_pm.append(id_pm)

            print(db_id_pm)
            select_gb_pm = f"SELECT Id_GB FROM GENBANK WHERE GB_ID= \"{loc_name_list[index_ids]}\""
            print(select_gb_pm)
            Cursor.execute(select_gb_pm)
            myresult_gb = Cursor.fetchall()
            id_gb = None
            for res, in myresult_gb:
                id_gb = res

            index = 0
            sql_pm_gb_relation = "INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES" 
            for id_pm in db_id_pm:
                if id_gb is not None and id_pm is not None:      
                    #verificar se existe relação entre ids
                    select_rel_gb_pm = f"SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = \"{id_pm}\" and GENBANK_Id_GB = \"{id_gb}\""
                    print(select_rel_gb_pm)
                    Cursor.execute(select_rel_gb_pm)
                    myresult_rel_pm_gb = Cursor.fetchall()
                    id_rel_pm_gb = None
                    for res_PUBMED_Id_PM,res_GENBANK_Id_GB, in myresult_rel_pm_gb:   #apenas selecionamos um dos id pq se não houver relação serão ambos null
                        id_rel_pm_gb = res_PUBMED_Id_PM     

                    #se null:
                    if id_rel_pm_gb is None:
                        if index == len(db_id_pm)-1:
                            sql_pm_gb_relation += f" ({id_pm}, {id_gb})"
                        else:
                            sql_pm_gb_relation += f" ({id_pm}, {id_gb}),"
                index += 1

            print(sql_pm_gb_relation)
            if sql_pm_gb_relation != "INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES":
                #remove ultimo caracter se este for uma ","
                if sql_pm_gb_relation.endswith(","):
                    sql_pm_gb_relation = sql_pm_gb_relation[:-1] #remove utimo caracter da string

                print(sql_pm_gb_relation)
                Cursor.execute(sql_pm_gb_relation)
                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

9
9
[1]
SELECT Id_GB FROM GENBANK WHERE GB_ID= "BW778499"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "1" and GENBANK_Id_GB = "2"
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (1, 2)
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (1, 2)
1 record inserted.
[1]
SELECT Id_GB FROM GENBANK WHERE GB_ID= "BW778502"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "1" and GENBANK_Id_GB = "3"
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (1, 3)
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (1, 3)
1 record inserted.
[2]
SELECT Id_GB FROM GENBANK WHERE GB_ID= "AAHY01223610"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM pm_gb_relationship WHERE PUBMED_Id_PM = "2" and GENBANK_Id_GB = "6"
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (2, 6)
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (2, 6)
1 record inserted.

In [690]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)

    for index_pubmed in range(len(pub_list)):
        
        affiliation_ids = []
        for index in range(len(affiliation[index_pubmed])):

            #verificar se existe afiliação (evitar duplicados)
            select_affiliation = f"SELECT Id_affiliation FROM AFFILIATION WHERE Description = \"{affiliation[index_pubmed][index]}\""
            Cursor.execute(select_affiliation)
            myresult_af = Cursor.fetchall()
            id_affiliation = None
            for res, in myresult_af:
                id_affiliation = res


            if id_affiliation is not None:
                affiliation_ids.append(id_affiliation)
            else:
                sql_insert_affiliation = "INSERT INTO AFFILIATION (Description) VALUES" 
                sql_insert_affiliation += f" (\"{affiliation[index_pubmed][index]}\")"
                print(sql_insert_affiliation)
                Cursor.execute(sql_insert_affiliation)
                # print(Cursor.lastrowid)  #buscar o id do affiliation inserido
                affiliation_ids.append(Cursor.lastrowid)


        sql = "INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES"
        
        for index in range(len(author[index_pubmed])):
            select_author = f"SELECT Id_Author FROM AUTHORS WHERE Name = \"{author[index_pubmed][index]}\""
            Cursor.execute(select_author)
            myresult_author = Cursor.fetchall()
            id_author = None
            for res, in myresult_author:
                id_author = res


            if id_author is None:
                if index > len(affiliation_ids)-1:
                    if index == len(author[index_pubmed])-1:
                        sql += f" (\"{author[index_pubmed][index]}\", null)"
                    else:
                        sql += f" (\"{author[index_pubmed][index]}\", null),"
                else:
                    if index == len(author[index_pubmed])-1:
                        sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]})"
                    else:
                        sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]}),"
    
        print(sql)
        if sql != "INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES":
                #remove ultimo caracter se este for uma ","
            if sql.endswith(","):
                sql = sql[:-1]

            Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO AFFILIATION (Description) VALUES ("Celera Genomics, 45 West Gude Drive, Rockville, MD 20850, USA. richard.mural@celera.com")
INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Mural RJ", 1), ("Adams MD", null), ("Myers EW", null), ("Smith HO", null), ("Miklos GL", null), ("Wides R", null), ("Halpern A", null), ("Li PW", null), ("Sutton GG", null), ("Nadeau J", null), ("Salzberg SL", null), ("Holt RA", null), ("Kodira CD", null), ("Lu F", null), ("Chen L", null), ("Deng Z", null), ("Evangelista CC", null), ("Gan W", null), ("Heiman TJ", null), ("Li J", null), ("Li Z", null), ("Merkulov GV", null), ("Milshina NV", null), ("Naik AK", null), ("Qi R", null), ("Shue BC", null), ("Wang A", null), ("Wang J", null), ("Wang X", null), ("Yan X", null), ("Ye J", null), ("Yooseph S", null), ("Zhao Q", null), ("Zheng L", null), ("Zhu SC", null), ("Biddick K", null), ("Bolanos R", null), ("Delcher AL", null), ("Dew IM", null), ("Fasulo D", null), ("Flanigan MJ", null), ("Hus

In [691]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `teste`.`PM_AUTHORS_relationship` (
        `PUBMED_Id_PM` INT NOT NULL,
        `AUTHORS_Id_Author` INT NOT NULL,
        PRIMARY KEY (`PUBMED_Id_PM`, `AUTHORS_Id_Author`)
    )
    """
    Cursor.execute(TableName)
    print(len(file_locus_pub_list))
    print(len(loc_name_list))

    for index_pm in range(len(pub_list)):

        if pub_list[index_pm] != '':
            select_pm_at = f"SELECT Id_PM FROM PUBMED WHERE PM_ID = \"{pub_list[index_pm]}\""
        
            Cursor.execute(select_pm_at)
            myresult_pm_at = Cursor.fetchall()
            id_pm_at = None
            for res, in myresult_pm_at:
                id_pm_at = res

            print(id_pm_at)
            author_id_list = []
            for xx in author[index_pm]:
                select_at_pm = f"SELECT Id_Author FROM AUTHORS WHERE Name= \"{xx}\""
                Cursor.execute(select_at_pm)
                myresult_at = Cursor.fetchall()
                id_at = None
                for res, in myresult_at:
                    id_at = res
                author_id_list.append(id_at)

            index = 0
            sql_at_pm_relation = "INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES" 
            author_id_list = list(dict.fromkeys(author_id_list))
            for id_at in author_id_list:
                if id_at is not None and id_pm_at is not None:      
                    #verificar se existe relação entre ids
                    select_rel_at_pm = f"SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = \"{id_pm_at}\" and AUTHORS_Id_Author = \"{id_at}\""
                    print(select_rel_at_pm)
                    Cursor.execute(select_rel_at_pm)
                    myresult_rel_pm_at = Cursor.fetchall()
                    id_rel_pm_at = None
                    for res_PUBMED_Id_PM,res_AUTHORS_Id_Author, in myresult_rel_pm_at:   #apenas selecionamos um dos id pq se não houver relação serão ambos null
                        id_rel_pm_at = res_PUBMED_Id_PM     

                    #se null:
                    if id_rel_pm_at is None:
                        if index == len(author_id_list)-1:
                            sql_at_pm_relation += f" ({id_pm_at}, {id_at})"
                        else:
                            sql_at_pm_relation += f" ({id_pm_at}, {id_at}),"
                index += 1

            print(sql_at_pm_relation)
            if sql_at_pm_relation != "INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES":
                #remove ultimo caracter se este for uma ","
                if sql_at_pm_relation.endswith(","):
                    sql_at_pm_relation = sql_at_pm_relation[:-1] #remove utimo caracter da string

                print(sql_at_pm_relation)
                Cursor.execute(sql_at_pm_relation)
                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

9
9
1
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "1"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "2"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "3"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "4"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "5"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "6"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "7"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "1" and AUTHORS_Id_Author = "8"
SELECT PUBMED_Id_PM, AUTHO

In [702]:
gene

[[['Oct 2']], [], [], [], [], [], [], [], []]

In [695]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `teste`.`FEATURES` (
        `Id_F` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Gene` VARCHAR(45) NULL,
        `Source_location` VARCHAR(45) NULL
    
    )
    """
    Cursor.execute(TableName)

    TableName ="""CREATE TABLE IF NOT EXISTS `teste`.`CDS` (
        `Id_CDS` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Translation_seq` longtext  NULL,
        `Location` VARCHAR(45)  NULL,
        `Product` VARCHAR(45)  NULL,
        `Protein_ID` VARCHAR(45) NULL
    )
    """
    Cursor.execute(TableName)

    for index_gb in range(len(loc_name_list)):

        #get genbank id
        select_gb = f"SELECT Id_GB FROM GENBANK WHERE GB_ID= \"{loc_name_list[index_gb]}\""
        Cursor.execute(select_gb)
        myresult_gb = Cursor.fetchall()
        id_gb = None
        for res, in myresult_gb:
            id_gb = res
        
        print(id_gb)
        if id_gb is not None:
            for index_ft in range(len(source[index_gb])):
                if source[index_gb][index_ft] != '':  #verificar se existe "source" na bd
                    sql_insert_source = "INSERT INTO FEATURES (Gene, Source_location) VALUES"
                    select_ft = f"SELECT Id_F FROM FEATURES WHERE Source_location = \"{source[index_gb][index_ft]}\""
            
                    Cursor.execute(select_ft)
                    myresult_ft = Cursor.fetchall()
                    id_ft = None
                    for res_ft, in myresult_ft:
                        id_ft = res_ft

                    if id_ft is None: #se n existe, insere
                        if len(gene[index_gb]) > 0:
                            sql_insert_source += f" (\"{gene[index_gb][index_ft]}\", \"{source[index_gb][index_ft]}\")"
                        else:
                            sql_insert_source += f" ( null, \"{source[index_gb][index_ft]}\")"

                        if sql_insert_source.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_source = sql_insert_source[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_source)

                        # print(Cursor.lastrowid)  #buscar o id do feature inserido
                        id_inserted_feature = Cursor.lastrowid
                    else:
                        id_inserted_feature = id_ft

                    #relationship between genbank and feature
                    sql_insert_gb_ft = "INSERT INTO GB_FT_relationship (GENBANK_Id_GB, FEATURES_Id_F) VALUES"
                    select_gb_ft = f"SELECT GENBANK_Id_GB FROM GB_FT_relationship WHERE GENBANK_Id_GB = {id_gb} and FEATURES_Id_F = {id_inserted_feature}"
            
                    Cursor.execute(select_gb_ft)
                    myresult_gb_ft = Cursor.fetchall()
                    id_gb_ft = None
                    for res_gb_ft, in myresult_gb_ft:
                        id_gb_ft = res_gb_ft

                    if id_gb_ft is None: #se n existe, insere
                        sql_insert_gb_ft += f" ({id_gb}, {id_inserted_feature})"

                        if sql_insert_gb_ft.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_gb_ft = sql_insert_gb_ft[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_gb_ft)

            for index_cds in range(len(CDS[index_gb])):
                print(CDS[index_gb])
                print(CDS[index_gb][index_cds])
                if CDS[index_gb][index_cds]:  #verificar se existe "cds" na bd
                    sql_insert_cds = "INSERT INTO CDS (Translation_seq, Location,Product,Protein_ID) VALUES"
                    select_cds = f"SELECT Id_CDS FROM CDS WHERE Translation_seq = \"{CDS[index_gb][index_cds][3]}\""
            
                    Cursor.execute(select_cds)
                    myresult_cds = Cursor.fetchall()
                    id_cds = None
                    for res_cds, in myresult_cds:
                        id_cds = res_cds

                    if id_cds is None: #se n existe, insere
                        sql_insert_cds += f" (\"{CDS[index_gb][index_cds][3]}\",\"{CDS[index_gb][index_cds][0]}\",\"{CDS[index_gb][index_cds][1]}\", \"{CDS[index_gb][index_cds][2]}\")"

                        if sql_insert_cds.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_cds = sql_insert_cds[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_cds)

                        # print(Cursor.lastrowid)  #buscar o id do cds inserido
                        id_inserted_cds = Cursor.lastrowid
                    else:
                        id_inserted_cds = id_cds
                    
                    #relationship between cds and genbank
                    sql_insert_gb_cds = "INSERT INTO CDS_GB_relationship (GENBANK_Id_GB, CDS_Id_CDS) VALUES"
                    select_gb_cds = f"SELECT GENBANK_Id_GB FROM CDS_GB_relationship WHERE GENBANK_Id_GB = {id_gb} and CDS_Id_CDS = {id_inserted_cds}"
            
                    Cursor.execute(select_gb_cds)
                    myresult_gb_cds = Cursor.fetchall()
                    id_gb_cds = None
                    for res_gb_cds, in myresult_gb_cds:
                        id_gb_ft = res_gb_cds

                    if id_gb_ft is None: #se n existe, insere
                        sql_insert_gb_cds += f" ({id_gb}, {id_inserted_cds})"

                        if sql_insert_gb_cds.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_gb_cds = sql_insert_gb_cds[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_gb_cds)

                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

1
[[], [], ['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'], ['FTQGDVGLAMGKLYGNDFSQTTISRFEALNLSFKNMCKLKPLLEKWLNDAETMSVDSSLPSPNQLSSPSLGFDGLPGRRRKKRTSIETNVRFALEKSFLANQKPTSEEILLIAEQLHMEKEVI']], [], [], []]
[]
1 record inserted.
[[], [], ['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'], ['FTQGDVGLAMGKLYGNDFSQTTISRFEALNLSFKNMCKLKPLLEKWLNDAETMSVDSSLPSPNQLSSPSLGFDGLPGRRRKKRTSIETNVRFALEKSFLANQKPTSEEILLIAEQLHMEKEVI']], [], [], []]
[]
1 record inserted.
[[], [], ['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'], ['FTQGDVGLAMGKLYGNDFSQTTISRFEALNLSFKNMCKLKPLLEKWLNDAETMSVDSSLPSPNQLSSPSLGFDGLPGRRRKKRTSIETNVRFALEKSFLANQKPTSEEILLIAEQLHMEKEVI']], [], [], []]
['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'], ['FTQGDVGLAMGKLYGNDFSQTTISRFEALNLSFKNMCKLKPLLEKWLNDAETMSVDSSLPSPNQLSSPSLGFDGLPGRRRKKRTSIETNVRFALEKSFLANQKPTSEEILLIAEQLHMEKEVI']]
1 record inserted.
[[], [], ['[<0:>369](+)', ['Oct 2 Pou-Homeo domain protein'], ['CAA47715.1'],